# BLR (Conjugate / g-prior)

#### Speed specifications & Imports

In [32]:
# Imports 
import numpy as np
import pandas as pd

from IPython.display import display
from joblib import Parallel, delayed, dump
from threadpoolctl import threadpool_limits

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, RegressorMixin

import warnings; warnings.filterwarnings("ignore")
import re
from math import sqrt

#  SPEED HEADER (single-BLAS + joblib CV) 
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["NUMEXPR_MAX_THREADS"] = "1"

# Parallelism knob 
N_JOBS = -1 # Use all available cores
RANDOM_STATE = 42

# Print fold-by-fold metrics?
VERBOSE_CV = False  #  False to parallelize configs with clean logs

#### Paths, Data Loading & Checks

In [33]:
SAVE_DIR   = 'Data+Files+Plots+etc'
TRAIN_CSV  = f'{SAVE_DIR}/train.csv'
TEST_CSV   = f'{SAVE_DIR}/test.csv'
FOLDS_NPY  = f'{SAVE_DIR}/train_folds.npy' 

# Output paths for residuals
RESID_DIR_OOF = "Data+Files+Plots+etc/Reports"  # OOF Residuals
os.makedirs(RESID_DIR_OOF, exist_ok=True)

def slug(obj):
    """Filename-safe tag for params."""
    if isinstance(obj, dict) and obj:
        parts = []
        for k in sorted(obj.keys()):
            v = obj[k]
            if isinstance(v, (float, np.floating)): v = float(v)
            parts.append(f"{k}={v}")
        s = "__".join(parts)
    else:
        s = str(obj) if obj not in (None, {}, []) else ""
    return re.sub(r"[^A-Za-z0-9._=-]+", "_", s).strip("_")

#  Load splits 
df_train = pd.read_csv(TRAIN_CSV)
df_train["_rowpos"] = np.arange(len(df_train), dtype=int)
df_test  = pd.read_csv(TEST_CSV)

#  Required columns check 
required_cols = [
    'PL','distance','frequency','c_walls','w_walls',
    'co2','humidity','pm25','pressure','temperature','snr'
]
missing = [c for c in required_cols if c not in df_train.columns or c not in df_test.columns]
if missing:
    raise ValueError(f"Missing required columns in train/test: {missing}")

#### Cross-Validation Folds

In [34]:
fold_assignments_full = np.load(FOLDS_NPY)  # vector aligned 

rowpos = df_train["_rowpos"].to_numpy(dtype=int)
if rowpos.max() >= len(fold_assignments_full):
    raise ValueError(
        f"[folds] Mismatch: train_folds.npy len={len(fold_assignments_full)} < max(_rowpos)={rowpos.max()}.\n"
        f"train.csv and train_folds.npy are out of sync."
    )
fold_assignments = fold_assignments_full[rowpos]

K = int(fold_assignments.max()) + 1
folds = [(np.where(fold_assignments != k)[0], np.where(fold_assignments == k)[0]) for k in range(K)]
print(f"[CV] Using saved folds (remapped) | K={K} | n_train={len(df_train)} | "
      f"fold sizes={[len(v) for _, v in folds]}")


[CV] Using saved folds (remapped) | K=5 | n_train=1663627 | fold sizes=[554543, 277271, 277271, 277271, 277271]


#### Physics, Features & Metrics

In [35]:
# Physics helpers 
d0 = 1.0  # reference distance in meters

def z_of_d(d):
    d = np.clip(d.astype(float), 1e-6, None)
    return 10.0 * np.log10(d / d0)

def f_term(f):
    f = np.clip(f.astype(float), 1e-12, None)
    return 20.0 * np.log10(f)

#  Features & targets 
raw_feats  = ['distance','frequency','c_walls','w_walls',
              'co2','humidity','pm25','pressure','temperature','snr']
target_col = 'PL'

Xtr_raw = df_train[raw_feats].copy()
ytr_pl  = df_train[target_col].astype(float).values
Xte_raw = df_test[raw_feats].copy()
yte_pl  = df_test[target_col].astype(float).values

# Friis adjustment: y_adj = PL - 20*log10(f)
ftr_tr, ftr_te = f_term(Xtr_raw['frequency'].values), f_term(Xte_raw['frequency'].values)
ytr_adj, yte_adj = ytr_pl - ftr_tr, yte_pl - ftr_te

# Linear feature map used by BLR
LIN_COLS = ['z_d','c_walls','w_walls','co2','humidity','pm25','pressure','temperature','snr']

Xtr_lin = pd.DataFrame({
    'z_d': z_of_d(Xtr_raw['distance'].values),
    'c_walls': Xtr_raw['c_walls'].values,
    'w_walls': Xtr_raw['w_walls'].values,
    'co2': Xtr_raw['co2'].values,
    'humidity': Xtr_raw['humidity'].values,
    'pm25': Xtr_raw['pm25'].values,
    'pressure': Xtr_raw['pressure'].values,
    'temperature': Xtr_raw['temperature'].values,
    'snr': Xtr_raw['snr'].values
}, columns=LIN_COLS).values

Xte_lin = pd.DataFrame({
    'z_d': z_of_d(Xte_raw['distance'].values),
    'c_walls': Xte_raw['c_walls'].values,
    'w_walls': Xte_raw['w_walls'].values,
    'co2': Xte_raw['co2'].values,
    'humidity': Xte_raw['humidity'].values,
    'pm25': Xte_raw['pm25'].values,
    'pressure': Xte_raw['pressure'].values,
    'temperature': Xte_raw['temperature'].values,
    'snr': Xte_raw['snr'].values
}, columns=LIN_COLS).values

# Metrics (PL-domain) 
def rmse_r2_on_PL(y_true_pl, y_pred_adj, fterm):
    y_pred_pl = y_pred_adj + fterm
    rmse = sqrt(((y_true_pl - y_pred_pl) ** 2).mean())
    ss_res = ((y_true_pl - y_pred_pl) ** 2).sum()
    ss_tot = ((y_true_pl - y_true_pl.mean()) ** 2).sum()
    r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else np.nan
    return rmse, r2

#### BLR Estimators (Conjugate & g-prior)

In [36]:
# BLR estimators (Conjugate) 
class FullBLRConjugate(BaseEstimator, RegressorMixin):
    """
    Conjugate Bayesian Linear Regression with Normal–Inverse-Gamma prior:
        beta | sigma^2 ~ N(beta0, sigma^2 V0),  sigma^2 ~ Inv-Gamma(a0, b0)
    - Works on adjusted target (y = PL - 20 log10 f).
    - Adds intercept internally (augments X with a column of ones).
    - Assumes you standardized X upstream if using spherical V0 (we do via StandardScaler).
    """
    def __init__(self, beta0=None, V0_scale=1e6, a0=1e-2, b0=1e-2):
        self.beta0 = beta0
        self.V0_scale = float(V0_scale)
        self.a0 = float(a0)
        self.b0 = float(b0)
        # learned
        self.beta_n_ = None
        self.Vn_ = None
        self.an_ = None
        self.bn_ = None

    def _augment(self, X):
        n = X.shape[0]
        return np.hstack([np.ones((n, 1)), X])

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float).reshape(-1)
        X_aug = self._augment(X)         # [n x (p+1)]
        n, d = X_aug.shape

        beta0 = np.zeros(d) if self.beta0 is None else np.asarray(self.beta0, dtype=float).reshape(-1)
        if beta0.shape[0] != d:
            raise ValueError("beta0 size mismatch.")

        # Prior covariance: V0 = V0_scale * I_d  (weakly informative on standardized X)
        V0_inv = np.eye(d) / self.V0_scale

        XtX = X_aug.T @ X_aug
        Vn_inv = V0_inv + XtX
        # small jitter for numerical stability
        Vn = np.linalg.inv(Vn_inv + 1e-12*np.eye(d))

        Xty = X_aug.T @ y
        beta_n = Vn @ (V0_inv @ beta0 + Xty)

        an = self.a0 + 0.5 * n
        # bn two equivalent forms; this one is numerically stable:
        bn = self.b0 + 0.5*( y @ y + beta0 @ (V0_inv @ beta0) - beta_n @ (Vn_inv @ beta_n) )

        self.beta_n_ = beta_n
        self.Vn_ = Vn
        self.an_ = an
        self.bn_ = float(bn)
        return self

    def predict(self, X, return_std=False):
        X = np.asarray(X, dtype=float)
        X_aug = self._augment(X)
        mean = X_aug @ self.beta_n_
        if not return_std:
            return mean
        # predictive variance for y: (bn/an) * (1 + x^T Vn x)
        pred_var = (self.bn_ / self.an_) * (1.0 + np.sum((X_aug @ self.Vn_) * X_aug, axis=1))
        pred_std = np.sqrt(np.maximum(pred_var, 0.0))
        return mean, pred_std

# BLR estimators ( g-prior) 
class BLR_GPrior(BaseEstimator, RegressorMixin):
    """
    Zellner g-prior on slopes (intercept gets flat/improper prior):
      beta = [beta0 (intercept); beta_s] ;  beta_s | sigma^2 ~ N(0, g sigma^2 (X'X)^{-1})
    - g_mode: 'uip' (g = n), or 'eb' (empirical Bayes from OLS R^2).
    - Works on adjusted target; adds intercept internally; expects standardized X.
    """
    def __init__(self, g_mode='uip', a0=1e-2, b0=1e-2, g_fixed=None):
        self.g_mode = str(g_mode)
        self.a0 = float(a0)
        self.b0 = float(b0)
        self.g_fixed = None if g_fixed is None else float(g_fixed)
        # learned
        self.beta_n_ = None
        self.Vn_ = None
        self.an_ = None
        self.bn_ = None
        self.g_ = None

    def _augment(self, X):
        n = X.shape[0]
        return np.hstack([np.ones((n, 1)), X])

    def _ols_fit(self, X_aug, y):
        # least squares solution
        beta_ols, *_ = np.linalg.lstsq(X_aug, y, rcond=None)
        yhat = X_aug @ beta_ols
        resid = y - yhat
        return beta_ols, yhat, resid

    def _choose_g(self, X_aug, y):
        n, d = X_aug.shape
        p = d - 1  # number of slopes
        if self.g_fixed is not None:
            return max(self.g_fixed, 1e-8)
        if self.g_mode.lower() == 'uip':
            return float(n)
        # empirical Bayes from OLS R^2
        beta_ols, yhat, resid = self._ols_fit(X_aug, y)
        tss = ((y - y.mean())**2).sum()
        rss = (resid**2).sum()
        R2 = 0.0 if tss <= 0 else max(0.0, 1.0 - rss / tss)
        R2 = min(R2, 1.0 - 1e-8)
        # g_hat = max( (R2/(1-R2)) * (n - p - 1), 1e-8 )
        g_hat = (R2 / (1.0 - R2)) * max(n - p - 1, 1.0)
        return float(np.clip(g_hat, 1e-8, 1e12))

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float).reshape(-1)
        X_aug = self._augment(X)  # [n x (p+1)]
        n, d = X_aug.shape
        p = d - 1

        g = self._choose_g(X_aug, y)
        self.g_ = g

        # Build prior precision: V0_inv = diag([0, (1/g)*XtX_slopes]) in augmented coords
        # Compute blocks
        one = X_aug[:, [0]]           # intercept column (ones)
        Z   = X_aug[:, 1:]            # slopes (standardized)
        XtX_11 = (one.T @ one)        # scalar [1x1] = n
        XtX_12 = (one.T @ Z)          # [1 x p]
        XtX_22 = (Z.T @ Z)            # [p x p]

        V0_inv = np.zeros((d, d), dtype=float)
        if p > 0:
            V0_inv[1:, 1:] = (1.0 / g) * XtX_22

        XtX = X_aug.T @ X_aug
        Vn_inv = V0_inv + XtX
        Vn = np.linalg.inv(Vn_inv + 1e-12*np.eye(d))

        Xty = X_aug.T @ y
        beta_n = Vn @ Xty  # prior mean = 0 (on slopes), flat on intercept

        an = self.a0 + 0.5 * n
        # bn form with prior precision:
        bn = self.b0 + 0.5*( y @ y - beta_n @ (Vn_inv @ beta_n) )

        self.beta_n_ = beta_n
        self.Vn_ = Vn
        self.an_ = an
        self.bn_ = float(bn)
        return self

    def predict(self, X, return_std=False):
        X = np.asarray(X, dtype=float)
        X_aug = self._augment(X)
        mean = X_aug @ self.beta_n_
        if not return_std:
            return mean
        pred_var = (self.bn_ / self.an_) * (1.0 + np.sum((X_aug @ self.Vn_) * X_aug, axis=1))
        pred_std = np.sqrt(np.maximum(pred_var, 0.0))
        return mean, pred_std

#### Pipeline Builders & Grids

In [37]:
# Pipeline builders 
def build_blr_conjugate(cfg):
    # Standardize X; conjugate BLR on adjusted target
    return make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
        FullBLRConjugate(beta0=None, V0_scale=cfg["V0_scale"], a0=cfg["a0"], b0=cfg["b0"])
    )

def build_blr_gprior(cfg):
    # Standardize X; g-prior BLR on adjusted target
    return make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
        BLR_GPrior(g_mode=cfg["g_mode"], a0=cfg["a0"], b0=cfg["b0"])
    )

# Grids 
conj_grid = [dict(V0_scale=v, a0=1e-2, b0=1e-2) for v in (1e2, 1e3, 1e4, 1e5, 1e6)]
gprior_grid = [dict(g_mode=m, a0=1e-2, b0=1e-2) for m in ("uip","eb")]

blr_specs = [
    ("BLR-Linear (Conjugate)", build_blr_conjugate, conj_grid),
    ("BLR-Linear (g-prior)",   build_blr_gprior,   gprior_grid),
]

#### Posterior Unscaling (Original Units)

In [38]:
#  Unscale BLR posterior to original units 
def unscale_blr_posterior(pipeline, feat_names):
    """
    Map posterior (intercept + standardized coefs) back to original feature units.
    Returns (names, mean_orig, cov_orig).
    """
    steps = pipeline.named_steps
    scaler = steps['standardscaler']
    est = steps['fullblrconjugate'] if 'fullblrconjugate' in steps else steps['blr_gprior']

    beta_std = est.beta_n_.copy()         # length p+1
    Vn_std   = est.Vn_.copy()             # (p+1)x(p+1)

    mu = scaler.mean_.astype(float)
    sig = scaler.scale_.astype(float)
    p = len(mu)

    # Transform matrix T from [intercept, beta_std] -> [intercept_orig, beta_orig]
    T = np.zeros((p+1, p+1), dtype=float)
    T[0,0] = 1.0
    T[0,1:] = -mu / sig
    for j in range(p):
        T[j+1, j+1] = 1.0 / sig[j]

    beta_orig = T @ beta_std
    Vn_orig = T @ Vn_std @ T.T

    names = ["Intercept"] + list(feat_names)
    return names, beta_orig, Vn_orig

#### CV sweep (folded TRAIN), select best config per BLR variant, refit, and score on TEST (kept in-memory)

In [39]:
# K-fold CV over each BLR model
blr_results = []
blr_residuals_list = []

K = int(np.max(fold_assignments)) + 1

# (train_idx, val_idx) per fold, derived directly from fold_assignments
folds = [(np.flatnonzero(fold_assignments != k),
          np.flatnonzero(fold_assignments == k)) for k in range(K)]

print(f"Running CV for {len(blr_specs)} BLR variants | K={len(folds)} folds | grid eval={'parallel...' if (N_JOBS != 1) else 'sequential...'}")

def eval_cfg_blr(factory, cfg, folds):
    tr_rmse_list, val_rmse_list, tr_r2_list, val_r2_list = [], [], [], []

    for tr_idx, val_idx in folds:
        X_tr, X_val = Xtr_lin[tr_idx], Xtr_lin[val_idx]
        y_tr, y_val = ytr_adj[tr_idx], ytr_adj[val_idx]
        ypl_tr, ypl_val = ytr_pl[tr_idx], ytr_pl[val_idx]
        f_tr,  f_val  = ftr_tr[tr_idx],  ftr_tr[val_idx]

        pipe = factory(cfg)
        with threadpool_limits(limits=1, user_api="blas"):
            pipe.fit(X_tr, y_tr)

        y_tr_pred = pipe.predict(X_tr)
        rmse_tr, r2_tr = rmse_r2_on_PL(ypl_tr, y_tr_pred, f_tr)
        tr_rmse_list.append(rmse_tr); tr_r2_list.append(r2_tr)

        y_val_pred = pipe.predict(X_val)
        rmse_val, r2_val = rmse_r2_on_PL(ypl_val, y_val_pred, f_val)
        val_rmse_list.append(rmse_val); val_r2_list.append(r2_val)

    return {
        "cfg": cfg,
        "rmse_train_mean": float(np.mean(tr_rmse_list)), "rmse_train_sd": float(np.std(tr_rmse_list)),
        "rmse_val_mean":   float(np.mean(val_rmse_list)), "rmse_val_sd":   float(np.std(val_rmse_list)),
        "r2_train_mean":   float(np.mean(tr_r2_list)),    "r2_train_sd":    float(np.std(tr_r2_list)),
        "r2_val_mean":     float(np.mean(val_r2_list)),   "r2_val_sd":      float(np.std(val_r2_list)),
    }

for name, factory, grid in blr_specs:
    if len(grid) == 1:
        grid_results = [eval_cfg_blr(factory, grid[0], folds)]
    else:
        grid_results = Parallel(n_jobs=N_JOBS, backend="threading", prefer="threads", verbose=0)(
            delayed(eval_cfg_blr)(factory, cfg, folds) for cfg in grid
        )

    best_res = min(grid_results, key=lambda r: r["rmse_val_mean"])
    best_cfg, best_cv = best_res["cfg"], {k: v for k, v in best_res.items() if k != "cfg"}

    final_pipe = factory(best_cfg)
    with threadpool_limits(limits=1, user_api="blas"):
        final_pipe.fit(Xtr_lin, ytr_adj)

    # Test performance (PL domain)
    try:
        yte_pred_adj, yte_pred_std = final_pipe.predict(Xte_lin, return_std=True)
    except TypeError:
        yte_pred_adj = final_pipe.predict(Xte_lin)
        yte_pred_std = None

    test_rmse, test_r2 = rmse_r2_on_PL(yte_pl, yte_pred_adj, ftr_te)

    # Residuals on TEST (in-memory)
    PL_pred  = yte_pred_adj + ftr_te
    resid_db = yte_pl - PL_pred

    model_tag = f"BLR_{name}"

    # Posterior means & std errors (original units)
    names_u, mean_u, cov_u = unscale_blr_posterior(final_pipe, LIN_COLS)
    se_u = np.sqrt(np.clip(np.diag(cov_u), 0.0, None))
    coef_tbl = pd.DataFrame({"mean": mean_u, "std_err": se_u}, index=names_u)

    blr_results.append({
        "model":      name,
        "best_cfg":   best_cfg,
        "cv":         best_cv,
        "test":       {"rmse": float(test_rmse), "r2": float(test_r2)},
        "final_pipe": final_pipe,
        "coef_tbl":   coef_tbl,
        "model_tag":  model_tag
    })

print("\nDone processing!")

Running CV for 2 BLR variants | K=5 folds | grid eval=parallel...



Done processing!


#### BLR CV summary (mean ± sd across folds), best model highlighted, then OOF residuals for BEST are saved.*

In [40]:
best_overall  = min(blr_results, key=lambda r: r["cv"]["rmse_val_mean"])
best_name     = best_overall["model"]
best_cfg      = best_overall["best_cfg"]
best_long_tag = best_overall["model_tag"]

rows = []
for r in blr_results:
    cfg = r.get("best_cfg", {}) if isinstance(r.get("best_cfg", {}), dict) else {"cfg": str(r.get("best_cfg"))}
    rows.append({
        "model":            f"BLR_{r['model']}",
        "is_best":          (r["model_tag"] == best_long_tag),
        "cv_rmse_val_mean": r["cv"]["rmse_val_mean"],
        "cv_rmse_val_sd":   r["cv"]["rmse_val_sd"],
        "cv_r2_val_mean":   r["cv"]["r2_val_mean"],
        "cfg":              slug(cfg),
    })

blr_cv_table = (pd.DataFrame(rows)
                .sort_values(["cv_rmse_val_mean", "cv_rmse_val_sd", "model"])
                .reset_index(drop=True))

display(blr_cv_table)

# BEST residuals on TEST 
best_pipe = best_overall["final_pipe"]
yte_pred_adj = best_pipe.predict(Xte_lin)

PL_pred_test = yte_pred_adj + ftr_te
resid_test   = yte_pl - PL_pred_test

blr_test_df = pd.DataFrame({
    "row_id":   np.arange(len(df_test), dtype=int),
    "PL_true":  yte_pl,
    "PL_pred":  PL_pred_test,
    "resid_db": resid_test
})

test_path = f"{SAVE_DIR}/Reports/residuals__BLR__BEST__test.csv"
blr_test_df.to_csv(test_path, index=False)
print(f"\n[TEST] Saved best BLR test residuals: {test_path}")

# OOF residuals for BEST (train-only)
factory_for_best = next(f for (n, f, g) in blr_specs if n == best_name)

y_pred_adj_oof = np.full(len(ytr_adj), np.nan, dtype=float)
for tr_idx, val_idx in folds:
    pipe = factory_for_best(best_cfg)
    with threadpool_limits(limits=1, user_api="blas"):
        pipe.fit(Xtr_lin[tr_idx], ytr_adj[tr_idx])
    y_pred_adj_oof[val_idx] = pipe.predict(Xtr_lin[val_idx])

mask = ~np.isnan(y_pred_adj_oof)

PL_pred_oof = y_pred_adj_oof[mask] + ftr_tr[mask]
resid_oof   = ytr_pl[mask] - PL_pred_oof

blr_oof_df = pd.DataFrame({
    "row_id":   np.arange(len(df_train), dtype=int)[mask],
    "fold":     fold_assignments.astype(int)[mask],
    "PL_true":  ytr_pl[mask],
    "PL_pred":  PL_pred_oof,
    "resid_db": resid_oof
})

oof_path = f"{SAVE_DIR}/Reports/residuals__BLR__BEST__oof.csv"
blr_oof_df.to_csv(oof_path, index=False)
print(f"\n[OOF] Saved best BLR OOF residuals: {oof_path}")

,model,is_best,cv_rmse_val_mean,cv_rmse_val_sd,cv_r2_val_mean,cfg
0,BLR_BLR-Linear (g-prior),True,8.241577,0.582734,0.805532,a0=0.01__b0=0.01__g_mode=uip
1,BLR_BLR-Linear (Conjugate),False,8.241577,0.582735,0.805532,V0_scale=100.0__a0=0.01__b0=0.01



[TEST] Saved best BLR test residuals: Data+Files+Plots+etc/Reports/residuals__BLR__BEST__test.csv

[OOF] Saved best BLR OOF residuals: Data+Files+Plots+etc/Reports/residuals__BLR__BEST__oof.csv


#### Test metrics table (final fit on full TRAIN, evaluated on held-out TEST)

In [41]:
test_rows = []
for res in blr_results:
    te = res["test"]
    test_rows.append({
        "Model": res["model"],
        "Test RMSE": float(te["rmse"]),
        "Test R2":   float(te["r2"]),
    })

test_blr_df = pd.DataFrame(test_rows)
display(test_blr_df)


,Model,Test RMSE,Test R2
0,BLR-Linear (Conjugate),8.453447,0.798423
1,BLR-Linear (g-prior),8.453448,0.798423


#### Coefficients Table

In [42]:
coef_blr_df = pd.concat(
    [res["coef_tbl"]["mean"].rename(res["model"]) for res in blr_results],
    axis=1
)

print("\nPosterior Means (original units) — BLR, linear basis ")
display(coef_blr_df)


Posterior Means (original units) — BLR, linear basis 


,BLR-Linear (Conjugate),BLR-Linear (g-prior)
Intercept,2.305528,2.305544
z_d,3.866506,3.866504
c_walls,6.830244,6.830240
w_walls,1.977092,1.977091
co2,-0.002355,-0.002355
humidity,-0.091712,-0.091712
pm25,-0.095295,-0.095295
pressure,-0.008045,-0.008045
temperature,-0.141028,-0.141028
snr,-2.034426,-2.034425
